In [1]:
# EDA
    
    # Missing B measurements
    # Satellite movements across sectors leads to very discontinuous data in sectors

# Feature Selection
    
    # Obtain datetime
    # Data split by satellite which took measurement
    
# Feature Engineering
    
    # Mean sampling daily
    # Filling missing values with interpolation
    # Scaling B
    # Apply timesteps to create dataset

# Model Training
    
    # LSTM

# Hyperparameter optimization

    # 1. Number of Epochs
    # 2. Batch Size
    # 3. Number of Neurons
    # 4. Dropout
    # 5. Layers
    # 6. Regularization
    # 7. Optimization Algorithm
    # 8. Loss Function
    # 9. Time Steps

In [2]:
# Load data

import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../data/raw/fff_Bw_1min.dat', delim_whitespace=True)
df.head()

In [9]:
#Adding column names to dataframe
df.columns = ['year', 'month', 'day', 'hour', 'minute', 
              'L', 'MLT', 'MLAT', 'AL Index', 
              'KP Index', 'Satellite Flag', 'B']

df.head()

,year,month,day,hour,minute,L,MLT,MLAT,AL Index,KP Index,Satellite Flag,B
0,2010,6,1,10,51,9.13,22.69,-1.43,-92,1,0.0,0.000765
1,2010,6,1,10,52,9.03,22.7,-1.44,-94,1,0.0,0.000699
2,2010,6,1,10,53,9.02,22.7,-1.45,-95,1,0.0,0.000786
3,2010,6,1,10,54,9.06,22.71,-1.46,-93,1,0.0,0.000748
4,2010,6,1,10,55,8.95,22.72,-1.48,-96,1,0.0,0.000933


In [10]:
# Converting year + month + day + hour + minute + second to datetime
datetime = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])

# Adding datetime column to dataframe
df.insert(0, "datetime", datetime, True)

df.head()

,datetime,year,month,day,hour,minute,L,MLT,MLAT,AL Index,KP Index,Satellite Flag,B
0,2010-06-01 10:51:00,2010,6,1,10,51,9.13,22.69,-1.43,-92,1,0.0,0.000765
1,2010-06-01 10:52:00,2010,6,1,10,52,9.03,22.7,-1.44,-94,1,0.0,0.000699
2,2010-06-01 10:53:00,2010,6,1,10,53,9.02,22.7,-1.45,-95,1,0.0,0.000786
3,2010-06-01 10:54:00,2010,6,1,10,54,9.06,22.71,-1.46,-93,1,0.0,0.000748
4,2010-06-01 10:55:00,2010,6,1,10,55,8.95,22.72,-1.48,-96,1,0.0,0.000933


In [15]:
df.shape[0]

1659514

In [11]:
# Dropping useless columns
df = df.drop(['year', 'month', 'day', 'hour', 'minute', 'AL Index', 'KP Index'], axis=1)

df.head()

,datetime,L,MLT,MLAT,Satellite Flag,B
0,2010-06-01 10:51:00,9.13,22.69,-1.43,0.0,0.000765
1,2010-06-01 10:52:00,9.03,22.7,-1.44,0.0,0.000699
2,2010-06-01 10:53:00,9.02,22.7,-1.45,0.0,0.000786
3,2010-06-01 10:54:00,9.06,22.71,-1.46,0.0,0.000748
4,2010-06-01 10:55:00,8.95,22.72,-1.48,0.0,0.000933


In [12]:
# saving the dataframe 
df.to_csv('../data/processed/df_with_datetime.csv', index=False)
df = pd.read_csv('../data/processed/df_with_datetime.csv')
df.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,datetime,L,MLT,MLAT,Satellite Flag,B
0,2010-06-01 10:51:00,9.13,22.69,-1.43,0.0,0.000765
1,2010-06-01 10:52:00,9.03,22.7,-1.44,0.0,0.000699
2,2010-06-01 10:53:00,9.02,22.7,-1.45,0.0,0.000786
3,2010-06-01 10:54:00,9.06,22.71,-1.46,0.0,0.000748
4,2010-06-01 10:55:00,8.95,22.72,-1.48,0.0,0.000933


In [14]:
df.shape[0]

1659514

In [16]:
# Checking if Satellite flag data is valid
df['Satellite Flag'].value_counts()

0.000000    482881
1.000000    461864
2.000000    459017
0.001010      2519
0.001040      2455
             ...  
0.201000         1
0.092600         1
0.236000         1
0.000587         1
0.097700         1
Name: Satellite Flag, Length: 2334, dtype: int64

In [17]:
# Converting satellite flag values to int
df['Satellite Flag'] = df['Satellite Flag'].astype(int)
df['Satellite Flag'].value_counts()

0    738633
1    461864
2    459017
Name: Satellite Flag, dtype: int64

In [20]:
# Splitting data into 3 frames for each satellite
# For satellite 0

df_ = df.loc[(df['Satellite Flag'] == 0)]
df_.to_csv('../data/processed/df_sat0.csv', index=False)
df_ = pd.read_csv('../data/processed/df_sat0.csv')
df_['Satellite Flag'].value_counts()

0    738633
Name: Satellite Flag, dtype: int64

In [21]:
# For satellite 1

df_ = df.loc[(df['Satellite Flag'] == 1)]
df_.to_csv('../data/processed/df_sat1.csv', index=False)
df_ = pd.read_csv('../data/processed/df_sat1.csv')
df_['Satellite Flag'].value_counts()

1    461864
Name: Satellite Flag, dtype: int64

In [22]:
# For satellite 2

df_ = df.loc[(df['Satellite Flag'] == 2)]
df_.to_csv('../data/processed/df_sat2.csv', index=False)
df_ = pd.read_csv('../data/processed/df_sat2.csv')
df_['Satellite Flag'].value_counts()

2    459017
Name: Satellite Flag, dtype: int64

In [3]:
# Select a satellite

df = pd.read_csv('../data/processed/df_sat0.csv')

In [4]:
# Sort by datetime
df = df.sort_values(by = 'datetime')
df.shape

(738633, 6)

In [6]:
# Plotting data for 1 satellite to see if there is a trend
import matplotlib.pyplot as plt
plt.plot(df['datetime'], df['B'])
#plt.xticks(rotation='vertical')

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7fdfd6b6ce18> (for post_execute):


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7fdfd6b7a7b8> (for post_execute):


KeyboardInterrupt: 